In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = requests.post("https://tasks.aidevs.pl/token/functions", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

print(response_json)

{'code': 0, 'msg': 'send me definition of function named addUser that require 3 params: name (string), surname (string) and year of birth in field named "year" (integer). Set type of function to "object"', 'hint1': "send this definition as correct JSON structure inside 'answer' field (as usual)"}


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "addUser",
            "description": "Add a user to the system",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "User's first name",
                    },
                    "surname": {
                        "type": "string",
                        "description": "User's last name",
                    },
                    "year": {
                        "type": "integer", 
                        "description": "User's year of birth",
                    },
                },
                "required": ["name", "surname", "year"],
            },
        },
    }
]
    
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "user", "content": "Daniel Obajtek is the CEO of Orlen S.A. and he was born in Myślenice on 2th. January 1976."}
    ],
    tools=tools,
    tool_choice="auto"
)
response_message = completion.choices[0].message
tool_calls = response_message.tool_calls
print(tool_calls)

[ChatCompletionMessageToolCall(id='call_3L43tR0nyct9GI77pBRhhen2', function=Function(arguments='{"name":"Daniel","surname":"Obajtek","year":1976}', name='addUser'), type='function')]


In [6]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=tools[0]["function"]))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
